## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-05-13-05-01 +0000,bbc,Mini-mart crime network a 'pull factor' for il...,https://www.bbc.com/news/articles/ced6wjy5qqjo...
1,2025-11-05-13-04-38 +0000,bbc,Britain sliding 'into economic crisis' over £8...,https://www.bbc.com/news/articles/c7v80n7mz2jo...
2,2025-11-05-13-01-07 +0000,nyt,"Shutdown enters 36th day, as stalemate in Cong...",https://www.nytimes.com/live/2025/11/05/us/tru...
3,2025-11-05-13-00-00 +0000,wsj,Motion Picture Trade Group Pans Instagram’s Us...,https://www.wsj.com/business/media/motion-pict...
4,2025-11-05-12-55-21 +0000,wapo,Mamdani’s global city clashes with Trump’s nat...,https://www.washingtonpost.com/world/2025/11/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2305/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,trump,50
44,new,35
37,mamdani,26
192,election,21
70,will,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
376,2025-11-04-17-18-25 +0000,cbc,"New York City's new mayor, Zohran Mamdani, cal...",https://www.cbc.ca/news/world/new-york-city-el...,237
135,2025-11-05-06-34-00 +0000,wsj,The Democratic Party won the New York City may...,https://www.wsj.com/politics/elections/five-ta...,228
162,2025-11-05-04-42-00 +0000,wsj,"Andrew Cuomo, part of New York’s political fir...",https://www.wsj.com/politics/elections/half-a-...,161
353,2025-11-04-18-37-45 +0000,latimes,Trump pushes hard against Mamdani as New Yorke...,https://www.latimes.com/politics/story/2025-11...,146
127,2025-11-05-07-26-00 +0000,wsj,Zohran Mamdani sealed his mayoral victory in N...,https://www.wsj.com/politics/elections/the-vot...,144


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
376,237,2025-11-04-17-18-25 +0000,cbc,"New York City's new mayor, Zohran Mamdani, cal...",https://www.cbc.ca/news/world/new-york-city-el...
135,83,2025-11-05-06-34-00 +0000,wsj,The Democratic Party won the New York City may...,https://www.wsj.com/politics/elections/five-ta...
123,76,2025-11-05-07-34-00 +0000,wsj,Lawmakers are hopeful they can reach a biparti...,https://www.wsj.com/politics/policy/lawmakers-...
162,60,2025-11-05-04-42-00 +0000,wsj,"Andrew Cuomo, part of New York’s political fir...",https://www.wsj.com/politics/elections/half-a-...
329,55,2025-11-04-19-43-00 +0000,wsj,The Trump administration will comply with a co...,https://www.wsj.com/politics/policy/trump-thre...
52,52,2025-11-05-11-00-00 +0000,nypost,"Eric Trump shares his dad’s family rules, talk...",https://nypost.com/2025/11/05/us-news/eric-tru...
128,50,2025-11-05-07-20-00 +0000,wsj,UPS Cargo Plane Crash in Kentucky Leaves at Le...,https://www.wsj.com/us-news/ups-cargo-plane-cr...
256,49,2025-11-04-23-11-18 +0000,cbc,Democratic candidates win key governor races a...,https://www.cbc.ca/news/world/virginia-new-jer...
323,48,2025-11-04-20-01-11 +0000,cbc,How budget day unfolded in Ottawa — from budge...,https://www.cbc.ca/news/politics/livestory/how...
214,46,2025-11-05-01-00-04 +0000,nypost,Democrat Abigail Spanberger wins VA gubernator...,https://nypost.com/2025/11/04/us-news/democrat...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
